In [50]:
# from ipyfilechooser import FileChooser
from netCDF4 import Dataset
import numpy.ma as ma
import numpy as np
from ipyleaflet import Polyline, Map, Marker, basemap_to_tiles, basemaps, AwesomeIcon

In [51]:
# Select Argo file
# projDir = 'Argo_data/'
# fc = FileChooser(projDir)
# display(fc)

In [52]:
# argo_fn = fc.selected
argo_fn = 'Argo_data//2902730_prof.nc'

In [53]:
# Open NetCDF Argo file
nc_fid = Dataset(argo_fn,'r')
vars = nc_fid.variables.keys()
# display(vars)

In [54]:
lat = nc_fid.variables['LATITUDE'][:]
lon = nc_fid.variables['LONGITUDE'][:]
dateJD = nc_fid.variables['JULD'][:]
date_ref = nc_fid.variables['REFERENCE_DATE_TIME'][:]
nc_fid.close()

In [55]:
# Reference datetime
dd=date_ref.tobytes()
dd=dd.decode('utf-8')
ddnp = np.datetime64(dd[0:4]+'-'+dd[4:6]+'-'+dd[6:8]+'T'+dd[8:10]+':'+dd[10:12]+':'+dd[12:14])
# print(ddnp)

In [56]:
date = ddnp + np.array(dateJD//1,dtype='timedelta64[D]') + np.array((dateJD%1)*24*60*60,dtype='timedelta64[s]')
# date

In [57]:
nProf = lat.count()
# nProf

In [58]:
latitude = lat.reshape(nProf,1)
longitude = lon.reshape(nProf,1)
coords = ma.concatenate([latitude, longitude],axis=1)
# display(coords)

In [59]:
argoTraj_line = Polyline(
    locations=coords.tolist(),
    color="black" ,
    fill=False,
    weight=3
)

In [62]:
from ipywidgets import Label
label = Label()
# display(label)
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        m_coords = kwargs.get('coordinates')
        m_coords[0] = round(m_coords[0],5)
        m_coords[1] = round(m_coords[1],5)
        label.value = str(m_coords[0])+'  '+str(m_coords[1])

#
center = (lat.mean(), lon.mean())
m = Map(basemap=basemap_to_tiles(basemaps.Esri.NatGeoWorldMap), center=center, zoom=5)
m.add_layer(argoTraj_line);
for i in range(0,nProf):
    marker = Marker(location = coords[i].tolist(),draggable=False)
    marker.title = 'Profile '+str(i+1)+'\n'+str(date[i])
    marker.rise_on_hover = True
    marker.icon = AwesomeIcon(name='share',marker_color='beige',icon_color='black',spin=False)
    m.add_layer(marker)
m.layout.width = '100%'
m.layout.height = '500px'
# m.on_interaction(handle_interaction)
display(m)

Map(center=[12.967607843137257, 127.63063725490196], controls=(ZoomControl(options=['position', 'zoom_in_text'…